# Classification : Signal events from background events

In [32]:
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install tensorflow-quantum==0.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py'>

## Data Preprocessing

In [35]:
import numpy as np

In [36]:
# Load data
dataset = []
dataset = np.load('QIS_EXAM_200Events.npz',allow_pickle = True)

In [37]:
for i in dataset.keys():
    print(i)

training_input
test_input


In [38]:
dataset # 0:Signal events 1:Background events

In [39]:
X_train = dataset['training_input'].item()['0']
y_train = dataset['training_input'].item()['1']

In [40]:
X_test = dataset['test_input'].item()['0']
y_test = dataset['test_input'].item()['1']

In [41]:
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score

In [42]:
import cirq

In [43]:
import sympy

In [44]:
# Define the qubits
qubits = [cirq.GridQubit(0, i) for i in range(4)]

# Convert each input feature to a circuit
X_train_circuits = []
for x in X_train:
    circuit = cirq.Circuit()
    for i, xi in enumerate(x):
        if xi == 1:
            circuit.append(cirq.X(qubits[i]))
    circuit.append(cirq.measure(*qubits, key='result'))
    X_train_circuits.append(circuit)

# Convert y_train to tensor
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Convert X_train to tensor
X_train_tensor = tfq.convert_to_tensor(X_train_circuits)

In [45]:
N_TRAIN = 5

In [46]:
X_train.shape

(50, 5)

In [47]:
X_train_tensor.shape

TensorShape([50])

In [48]:
N_QUBITS = 5

In [49]:
LATENT_DIM = 50
def make_generator_model():
    """Construct generator model."""
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(LATENT_DIM,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(N_QUBITS, activation='relu'))

    return model

def make_discriminator_model():
    """Constrcut discriminator model."""
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(N_QUBITS,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))

    return model

In [50]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [51]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    """Compute discriminator loss."""
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    """Compute generator loss."""
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [52]:
BATCH_SIZE=256

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(
        gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(
        disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(
        zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [53]:
import time

In [54]:
def train(dataset, epochs, start_epoch=1):
    """Launch full training run for the given number of epochs."""
    # Log original training distribution.
    batched_data = tf.data.Dataset.from_tensor_slices(dataset).shuffle(N_TRAIN).batch(512)
    t = time.time()
    for epoch in range(start_epoch, start_epoch + epochs):
        for i, image_batch in enumerate(batched_data):
            # Log batch-wise loss.
            gl, dl = train_step(image_batch)
            tf.summary.scalar(
                'Generator loss', data=gl, step=epoch * len(batched_data) + i)
            tf.summary.scalar(
                'Discriminator loss', data=dl, step=epoch * len(batched_data) + i)

        if epoch % 10 == 0:
            print('Epoch {}, took {}(s)'.format(epoch, time.time() - t),'Generator Loss : ',gl.numpy(),'Discriminator Loss :',dl.numpy())
            t = time.time()

In [55]:
train(X_train, epochs=200)

Epoch 10, took 2.6762290000915527(s) Generator Loss :  0.6851231 Discriminator Loss : 1.2971206
Epoch 20, took 0.15517330169677734(s) Generator Loss :  0.71206796 Discriminator Loss : 1.221551
Epoch 30, took 0.15167617797851562(s) Generator Loss :  0.73501706 Discriminator Loss : 1.1479139
Epoch 40, took 0.214141845703125(s) Generator Loss :  0.7397375 Discriminator Loss : 1.0975152
Epoch 50, took 0.28757476806640625(s) Generator Loss :  0.751351 Discriminator Loss : 1.0443809
Epoch 60, took 0.28740906715393066(s) Generator Loss :  0.7538519 Discriminator Loss : 1.0030103
Epoch 70, took 0.25377988815307617(s) Generator Loss :  0.73877764 Discriminator Loss : 0.98648816
Epoch 80, took 0.25180578231811523(s) Generator Loss :  0.7376641 Discriminator Loss : 0.935309
Epoch 90, took 0.2774319648742676(s) Generator Loss :  0.73420787 Discriminator Loss : 0.9083604
Epoch 100, took 0.26679372787475586(s) Generator Loss :  0.73838794 Discriminator Loss : 0.85971886
Epoch 110, took 0.23462224006